In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
!pwd

In [ ]:
!git clone https://github.com/at-tyagi/disease-classification

In [ ]:
IMAGE_SIZE=256 #pixels
BATCH_SIZE=32
CHANNELS=3

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "PlantVillage",  # give root directory name
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)



In [ ]:
class_names=dataset.class_names
print(class_names, len(dataset))
# 68 because batch of 32 for total 68 in no.
# use len() in place of shape
print(dataset)

In [ ]:
for es, ts in dataset.take(1):
    print(es.shape,ts.shape)
    print(ts.numpy()) # ts in tensor initially
    print(es[0])

In [ ]:
## 1,2,3 are classes
## es[0] is one image data 256x256x3
plt.figure(figsize=(10,10))
for img,label in dataset.take(1): ## img mapped to 32 img, label in one step only no loop again and again
    for i in range(10):# select ith from that 32 label
        ax=plt.subplot(3,4,i+1) ## i tell position 3,4 rows and columns
        ## plt.imshow(es[1].numpy()) give raw unbalancedcolours 
        plt.imshow(img[i].numpy().astype("uint8"))
        plt.title(class_names[label[i]])
        #print(img[0][0].numpy().astype("uint8"))   ### 256x3 shape
        plt.axis("off")
    print("only one print hence outer loop is one step process")

In [ ]:
## 80% , 10% , 10% splitting.........................................................
dataset=dataset.shuffle(1000,seed=12)
train_ds=dataset.take(int(len(dataset)*0.8))   ## [0:54]
test=dataset.skip(int(len(dataset)*0.8))    ## [54:end]
cv_ds=test.take(int(len(test)*0.5))    ## [0:mid]
test_ds=test.skip(int(len(test)*0.5))    ## [mid:end]
print(len(train_ds),len(test),len(cv_ds),len(test_ds))

In [ ]:
# ----------train_ds.cache()------------ ### read data from disk and kepp in memory so make process fast for next iteration [[[don't read same image again and again]]]
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)   ## prefetch load next batch if cpu/gpu busy in training in old batch
cv_ds=cv_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)   ## prefetch load next batch if cpu/gpu busy in training in old batch
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)   ## prefetch load next batch if cpu/gpu busy in training in old batch

In [ ]:
### now training will run fast
resize_and_scale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [ ]:
data_augmentation=tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

In [ ]:
## both made above are layers we will use furthur
input_shape=(BATCH_SIZE,IMAGE_SIZE, IMAGE_SIZE,CHANNELS)
n_classes=3
model=models.Sequential([
    resize_and_scale,
    data_augmentation,
    layers.Conv2D(32,(3,3),activation='relu',input_shape=input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3), activation='relu'),    
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3), activation='relu'),    
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3), activation='relu'),    
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3), activation='relu'),    
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(n_classes,activation='softmax')
])

In [ ]:
model.build(input_shape=input_shape)
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history=model.fit(
    train_ds,
    epochs=10,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=cv_ds
)
    

In [ ]:
scores=model.evaluate(test_ds)

In [ ]:
history

In [ ]:
history.params

In [ ]:
history.history.keys()

In [ ]:
 history.history['accuracy']

In [ ]:
plt.plot(range(10),history.history['accuracy'],label='accuracy')
plt.legend()
plt.show()
plt.plot(range(10),history.history['loss'],label='loss')
plt.legend()
plt.show()
plt.plot(range(10),history.history['val_accuracy'],label='val accuracy')
plt.legend()
plt.show()
plt.plot(range(10),history.history['val_loss'],label='val loss')
plt.legend()
plt.show()


In [ ]:
# ###########################                  image_batch , label batch
plt.figure(figsize=(15,15))
for ib , lb in test_ds.take(1):
    for i in range(10):
      ax=plt.subplot(5,2,i+1)
      plt.imshow(ib[i].numpy().astype('uint8'))
      plt.axis("off")
      a=class_names[lb[i]]
      b=np.max(model.predict(ib[i:i+1]))  # a 1x3 array with probablities
      d=np.round(b*100, decimals = 2)
      c=class_names[np.argmax(model.predict(ib[i:i+1]))]
      plt.title(f"actual: {a},\n Predicted: {c} \n Confidence: {d}")


In [ ]:
model_version=1
# model.save('path') method to save keras model
#using python formatted string
model.save(f"./models/{model_version}")

In [ ]:
## make change in model and save new version
# automatic update version
import os
lst=os.listdir('./models')  ####### string 
new_version=max([int(i) for i in lst])+1
print(new_version)